<a href="https://colab.research.google.com/github/kazut98/me315/blob/main/ME315_Lab02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Laboratório 02 - Processamento de Bases de Dados em Lote#

Começamos por configurar o ambiente de execução


In [50]:
## Instalação r2u

shell_call <- function(command, ...) {
  result <- system(command, intern = TRUE, ...)
  cat(paste0(result, collapse = "\n"))
}

download.file("https://github.com/eddelbuettel/r2u/raw/master/inst/scripts/add_cranapt_jammy.sh",
              "add_cranapt_jammy.sh")
Sys.chmod("add_cranapt_jammy.sh", "0755")
shell_call("./add_cranapt_jammy.sh")
bspm::enable()
options(bspm.version.check=FALSE)
shell_call("rm add_cranapt_jammy.sh")

## Preparando acesso ao Google Drive
loadPackages = function(pkgs){
  myrequire = function(...){
    suppressWarnings(suppressMessages(suppressPackageStartupMessages(require(...))))
  }
  ok = sapply(pkgs, require, character.only=TRUE, quietly=TRUE)
  if (!all(ok)){
    message("There are missing packages: ", paste(pkgs[!ok], collapse=", "))
    install.packages(pkgs[!ok])
    loadPackages(pkgs[!ok])
  }
}

loadPackages(c("googledrive", "httpuv", "R.utils", "httr"))

my_check = function() TRUE
reassignInPackage("is_interactive", pkgName="httr", my_check)
options(rlang_interactive=TRUE)

136 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists...
Building dependency tree...
Reading state information...
wget is already the newest version (1.21.2-2ubuntu1).
ca-certificates is already the newest version (20230311ubuntu0.22.04.1).
gnupg is already the newest version (2.2.27-3ubuntu2.1).
The following package was automatically installed and is no longer required:
  r-cran-svglite
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 136 not upgraded.
-----BEGIN PGP PUBLIC KEY BLOCK-----

...
I5rGSOk715L9
=VuA5
-----END PGP PUBLIC KEY BLOCK-----
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:6 https://developer.download.nvidia.c

Tracing function "install.packages" in package "utils"



Autorização ao acesso ao Google Drive

In [51]:
drive_auth(use_oob=TRUE, cache=FALSE)

Please point your browser to the following url: 

https://accounts.google.com/o/oauth2/v2/auth?client_id=603366585132-frjlouoa3s2ono25d2l9ukvhlsrlnr7k.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email&redirect_uri=https%3A%2F%2Fwww.tidyverse.org%2Fgoogle-callback%2F&response_type=code&state=d8a5ab37911bd3dea4f364ff3703aedc&access_type=offline&prompt=consent



Download do arquivo usando o pacote `googledrive` no R.

In [52]:
drive_download(as_id("1CZjtI4JpxLHqgztrvJhYIV9HmHkrFl-1"))

Auto-refreshing stale OAuth token.



ERROR: ignored

In [6]:
list.files()

[1] "flights.csv.zip" "sample_data"

Filtrando os dados

In [11]:
library("tidyverse")

In [29]:
conteudo100k = read_csv("flights.csv.zip", n_max=100000)

conteudo100k %>%
   filter(AIRLINE %in% c("AA", "DL", "US", "UA")) %>%
   select(YEAR, MONTH, DAY, AIRLINE, ARRIVAL_DELAY) %>%
     drop_na() %>%
     group_by(YEAR, MONTH, DAY, AIRLINE) %>%
     summarise(n_atrasos = sum(ARRIVAL_DELAY > 10), n_voos = n())



Rows: 100000 Columns: 31
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (11): AIRLINE, TAIL_NUMBER, ORIGIN_AIRPORT, DESTINATION_AIRPORT, SCHEDUL...
dbl (20): YEAR, MONTH, DAY, DAY_OF_WEEK, FLIGHT_NUMBER, DEPARTURE_DELAY, TAX...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
`summarise()` has grouped output by 'YEAR', 'MONTH', 'DAY'. You can override
using the `.groups` argument.


YEAR,MONTH,DAY,AIRLINE,n_atrasos,n_voos
<dbl>,<dbl>,<dbl>,<chr>,<int>,<int>
2015,1,1,AA,514,1379
2015,1,1,DL,132,1558
2015,1,1,UA,367,1269
2015,1,1,US,201,1028
2015,1,2,AA,649,1508
2015,1,2,DL,406,2261
2015,1,2,UA,493,1411
2015,1,2,US,229,1174
2015,1,3,AA,767,1425


In [ ]:
View(conteudo100k)

In [30]:
getStats = function(input, pos){
input %>% filter(AIRLINE %in% c("AA", "DL", "US", "UA")) %>%
   select(YEAR, MONTH, DAY, AIRLINE, ARRIVAL_DELAY) %>%
     drop_na() %>%
     group_by(YEAR, MONTH, DAY, AIRLINE) %>%
     summarise(n_atrasos = sum(ARRIVAL_DELAY > 10), n_voos = n())
}

In [ ]:
suficientes = read_csv_chunked("flights.csv.zip",
 callback = DataFrameCallback$new(getStats), chunk_size = 100000)
 #ineficiente, lê tudo e joga o resto fora
  #callback cria um dataframe a partir de tds tabelas criadas por getstats


In [42]:
minhas_col = cols_only(YEAR = "i", MONTH = "i", DAY = "i",
AIRLINE = "c", ARRIVAL_DELAY = "i")
suficientes = read_csv_chunked("flights.csv.zip",
 callback = DataFrameCallback$new(getStats), chunk_size = 100000,
 col_types = minhas_col) #mais eficiente, apenas lê as colunas do interesse, o getstats não precisa mais do filter

`summarise()` has grouped output by 'YEAR', 'MONTH', 'DAY'. You can override
using the `.groups` argument.


|==                                                                  | 3% 19 MB

`summarise()` has grouped output by 'YEAR', 'MONTH', 'DAY'. You can override
using the `.groups` argument.


|===                                                                 | 5% 28 MB

`summarise()` has grouped output by 'YEAR', 'MONTH', 'DAY'. You can override
using the `.groups` argument.


...

`summarise()` has grouped output by 'YEAR', 'MONTH', 'DAY'. You can override
using the `.groups` argument.


|==================================================================| 99% 563 MB

`summarise()` has grouped output by 'YEAR', 'MONTH', 'DAY'. You can override
using the `.groups` argument.


|==================================================================| 100% 564 MB

`summarise()` has grouped output by 'YEAR', 'MONTH', 'DAY'. You can override
using the `.groups` argument.


In [40]:
suficientes %>% filter(YEAR == 2015, MONTH == 3, DAY == 7, AIRLINE == "AA")
#duplicado pois tinha dois chunks com o mesmo valor

YEAR,MONTH,DAY,AIRLINE,n_atrasos,n_voos
<dbl>,<dbl>,<dbl>,<chr>,<int>,<int>
2015,3,7,AA,74,359
2015,3,7,AA,356,1042


In [39]:
suficientes %>% filter(YEAR == 2015, MONTH == 3, DAY == 7, AIRLINE == "AA") %>%
  group_by(YEAR, MONTH, DAY, AIRLINE) %>%
  summarise(n_total_atrasos = sum(n_atrasos), n_total_voos = sum(n_voos),
  p = n_total_atrasos/n_total_voos) #tira-se a média calculando as estatísticas suficientes

`summarise()` has grouped output by 'YEAR', 'MONTH', 'DAY'. You can override
using the `.groups` argument.


YEAR,MONTH,DAY,AIRLINE,n_total_atrasos,n_total_voos,p
<dbl>,<dbl>,<dbl>,<chr>,<int>,<int>,<dbl>
2015,3,7,AA,430,1401,0.3069236


In [47]:
atrasos = suficientes %>%
  group_by(YEAR, MONTH, DAY, AIRLINE) %>%
  summarise(n_total_atrasos = sum(n_atrasos), n_total_voos = sum(n_voos),
  p = n_total_atrasos/n_total_voos) #tira o filter e generaliza

`summarise()` has grouped output by 'YEAR', 'MONTH', 'DAY'. You can override
using the `.groups` argument.


#Criando calendário#

Criar calendário utilizando `devtools` e `ggcal`.

In [43]:
install.packages("devtools")

In [44]:
devtools::install_github("jayjacobs/ggcal")

isoband (0.2.6 -> 0.2.7) [CRAN]
gtable  (0.3.3 -> 0.3.4) [CRAN]
ggplot2 (3.4.2 -> 3.4.3) [CRAN]


Installing 3 packages: isoband, gtable, ggplot2



── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/tmp/Rtmp78kCOZ/remotes1e85f227d06/jayjacobs-ggcal-ab1a85a/DESCRIPTION’ ... OK
* preparing ‘ggcal’:
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
Omitted ‘LazyData’ from DESCRIPTION
* building ‘ggcal_0.1.0.tar.gz’



Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [45]:
library("ggcal")

In [49]:
atrasos %>%
  mutate(data = as.Date(paste(YEAR, MONTH, DAY, sep="-")))
atrasos_AA = atrasos %>% filter(AIRLINES == "AA")
grafico = ggcal(atrasos_AA$data, atrasos_AA$p)
ggsave("calendarioAA.pdf")

YEAR,MONTH,DAY,AIRLINE,n_total_atrasos,n_total_voos,p,data
<int>,<int>,<int>,<chr>,<int>,<int>,<dbl>,<date>
2015,1,1,AA,514,1379,0.37273387,2015-01-01
2015,1,1,DL,132,1558,0.08472401,2015-01-01
2015,1,1,UA,367,1269,0.28920410,2015-01-01
2015,1,1,US,201,1028,0.19552529,2015-01-01
2015,1,2,AA,649,1508,0.43037135,2015-01-02
2015,1,2,DL,406,2261,0.17956656,2015-01-02
2015,1,2,UA,493,1411,0.34939759,2015-01-02
2015,1,2,US,229,1174,0.19505963,2015-01-02
2015,1,3,AA,767,1425,0.53824561,2015-01-03


ERROR: ignored